In [6]:
import os
import sys
import json
import argparse
from typing import *
from tqdm import *
import numpy as np
import matplotlib.pyplot as plt

sys.path.append("..")
try:
  from ..utils.logger import logger
  from ..utils.utils import *
except ImportError:
  from utils.logger import logger
  from utils.utils import *

In [5]:
import numpy as np
from scipy import integrate
#import matplotlib.pyplot as plt
import sympy

def apply_ics(sol, ics, x, known_params):
	free_params = sol.free_symbols - set(known_params)
    ## free parameters like C1,C2, that needed to be figured out with ics and boundary conditions
	eqs = [(sol.lhs.diff(x, n) - sol.rhs.diff(x, n)).subs(x, 0).subs(ics) for n in range(len(ics))]
    ## form equations with general solution(sol), by substituting the x with 0 and [y(0),dy/dx(x=0)...] with ics.
	sol_params = sympy.solve(eqs, free_params)
    ## solve the algebraic equation set  to get the free parameters expression
    ## return the solution after substituting the free parameters

	return sol.subs(sol_params)

sympy.init_printing()
## initialize the print environment
t = sympy.Symbol("t")
## symbolize the parameters and they can only be positive
x1 = sympy.Function('x_1')
x2 = sympy.Function('x_2')
## set x to be the differential function, not the variable
f1 = 80000 * (x1(t) - x2(t)) + 10000 * (x1(t).diff(t) - x2(t).diff(t)) - x2(t).diff(t, 2) * 2433
f2 = 80000 * (x1(t) - x2(t)) + 10000 * (x1(t).diff(t) - x2(t).diff(t)) - 6250 * sympy.cos(1.0045 * t) * x2(t).diff(t, 2) + \
        1e3 * 9.8 * sympy.pi * x1(t) + 656.3616 * x1(t).diff(t) + 1335.535 * x1(t).diff(t, 2) + 4866 * x1(t).diff(t, 2)
sol = sympy.dsolve(f1, f2)
## use diff() and dsolve to get the general solution
ics = {x1(0): 0, x1(t).diff(t).subs(t, 0): 0, x2(0): 0, x2(t).diff(t).subs(t, 0): 0}
## apply dict to the initial conditions
t_sol = apply_ics(solsol, ics, t, [])
## get the solution with ics by calling function apply_ics.
sympy.pprint(t_sol)
print(t_sol)
## pretty print
## solution is as follows

ValueError: dsolve() and classify_ode() only work with functions of one variable, not 9800.0*pi*x_1(t) + 80000*x_1(t) - 80000*x_2(t) - 6250*cos(1.0045*t)*Derivative(x_2(t), (t, 2)) + 10656.3616*Derivative(x_1(t), t) + 6201.535*Derivative(x_1(t), (t, 2)) - 10000*Derivative(x_2(t), t)